# `목적`: Conversion Window 시각화

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from datetime import datetime, timedelta

### 데이터 로드

In [2]:
df = pd.read_csv(r'data-files\08-conversion.csv')
df.head()

,transaction_id,apply_at,pay_at
0,2560,2023-01-01,2023-01-08
1,2844,2023-01-01,2023-01-06
2,1878,2023-01-01,NaN
3,2374,2023-01-01,2023-01-02
4,3923,2023-01-01,2023-01-08


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2170 entries, 0 to 2169
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   transaction_id  2170 non-null   int64 
 1   apply_at        2170 non-null   object
 2   pay_at          1062 non-null   object
dtypes: int64(1), object(2)
memory usage: 51.0+ KB


In [4]:
# 날짜 데이터 타입 변환
df['apply_at'] = pd.to_datetime(df['apply_at'])
df['pay_at'] = pd.to_datetime(df['pay_at'])

In [5]:
# apply 와 pay 사이 시간간격 분포 (하루기준 계산)
df['td'] = (df['pay_at'] - df['apply_at']).dt.total_seconds() / (3600*24)

In [8]:
df.head()

,transaction_id,apply_at,pay_at,td
0,2560,2023-01-01,2023-01-08,7.0
1,2844,2023-01-01,2023-01-06,5.0
2,1878,2023-01-01,NaT,NaN
3,2374,2023-01-01,2023-01-02,1.0
4,3923,2023-01-01,2023-01-08,7.0


### 전환 시간 분포 시각화

In [52]:
# Conversion Window = 10일 설정
thread = 10

# 히스토그램 생성
fig = px.histogram(
    df
    , x = 'td'
    , title = 'Conversion Time Histogram'
    , height = 600
    , width = 1200
)

# 10일 이내 전환 하이라이트 및 주석 추가
fig.add_vrect(x0=-.5
              , x1=thread
              , fillcolor = 'red'
              , opacity = .2
              )

a = df[~df['pay_at'].isna()].shape[0]   # 구매 전환 수
b = df[df['td'] <= thread].shape[0]     # 10일 이내 구매 전환 수
ratio = b / a   # 전체 구매 전환 중 10일 이내 전환의 비율 계산

fig.add_annotation(x=thread
                   , text=f'CW: {thread}일 (전환의 {ratio*100:.1f}%포함)'
                   , font=dict(
                       color='black'
                       , size=14
                   )
                   )

# 전환 시간 분포의 10% 부터 90% 지점 표시
for perc in list(range(10, 100, 10)):
    lis = df[~df['td'].isna()]['td']
    dates = np.percentile(lis, perc)
    
    fig.add_vline(x=dates, line_color='gray', opacity=.7, line_dash='dash')
    fig.add_annotation(x=dates, y=100-perc, text=f'{perc}%: {dates:.0f}일')

fig.show()